# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from config import gkey



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/weather_data.csv")

In [3]:
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,chernyshevskiy,95,RU,1583329092,98,63.01,112.47,254.92,1.60
1,hofn,75,IS,1583329345,100,64.25,-15.21,272.15,2.60
2,talnakh,75,RU,1583329931,85,69.49,88.40,265.15,11.00
3,pangai,6,TO,1583329932,77,-19.80,-174.35,299.54,5.11
4,coari,100,BR,1583329803,77,-4.08,-63.14,301.18,1.84


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=gkey)

In [5]:
geocode = weather_data[["Lat","Lng"]]
humidity = weather_data["Humidity"].astype(float)
print(geocode.head())


     Lat     Lng
0  63.01  112.47
1  64.25  -15.21
2  69.49   88.40
3 -19.80 -174.35
4  -4.08  -63.14


In [6]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(geocode, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
weather_data["Max_TempF"] = (weather_data["Max_Temp"]-273) * (9/5) + 32


In [8]:
vacation_weather = weather_data.loc[(weather_data["Max_TempF"]<80)&(weather_data["Max_TempF"]>70)]

In [9]:
vacation_weather = vacation_weather.loc[(vacation_weather["Cloudiness"]==0)]

In [10]:
vacation_weather = vacation_weather.loc[(vacation_weather["Humidity"]<50)]

In [11]:
vacation_weather = vacation_weather.loc[(vacation_weather["Wind_Speed"]<4.4704)]

In [12]:
vacation_weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Max_TempF
42,yulara,0,AU,1583329950,38,-25.24,130.99,296.15,2.60,73.670
171,constitucion,0,CL,1583330004,42,-35.33,-72.42,295.11,4.41,71.798
188,riyadh,0,SA,1583329399,16,24.69,46.72,299.15,3.60,79.070
244,castro,0,BR,1583330033,42,-24.79,-50.01,297.99,1.41,76.982
316,buraydah,0,SA,1583330063,27,26.33,43.98,298.15,3.60,77.270
382,ambagarh chauki,0,IN,1583330107,46,20.78,80.72,297.54,2.01,76.172
422,chilmari,0,BD,1583330126,42,25.56,89.67,297.12,1.25,75.416
465,araria,0,IN,1583330145,33,26.15,87.52,297.22,2.06,75.596
488,alirajpur,0,IN,1583330157,26,22.30,74.35,296.10,2.95,73.580
567,sakakah,0,SA,1583329897,12,29.97,40.21,299.15,2.10,79.070


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = vacation_weather

In [14]:
hotel_df["HotelName"] = ""

In [15]:
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Max_TempF,HotelName
42,yulara,0,AU,1583329950,38,-25.24,130.99,296.15,2.60,73.670,
171,constitucion,0,CL,1583330004,42,-35.33,-72.42,295.11,4.41,71.798,
188,riyadh,0,SA,1583329399,16,24.69,46.72,299.15,3.60,79.070,
244,castro,0,BR,1583330033,42,-24.79,-50.01,297.99,1.41,76.982,
316,buraydah,0,SA,1583330063,27,26.33,43.98,298.15,3.60,77.270,


In [16]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    params = {
        "radius": 5000,
        "types": "hotel",
        "key": gkey,
        "location": f"{lat},{lng}"
    }

    request_url = requests.get(base_url, params=params)
    response = request_url.json()
    
    nearest_hotel = response["results"][1]["name"]
    
    hotel_df.loc[index,"HotelName"] = nearest_hotel

In [17]:
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Max_TempF,HotelName
42,yulara,0,AU,1583329950,38,-25.24,130.99,296.15,2.60,73.670,Desert Gardens Hotel - Ayers Rock Resort
171,constitucion,0,CL,1583330004,42,-35.33,-72.42,295.11,4.41,71.798,Hotel Boutique Las Azucenas
188,riyadh,0,SA,1583329399,16,24.69,46.72,299.15,3.60,79.070,InterContinental Riyadh
244,castro,0,BR,1583330033,42,-24.79,-50.01,297.99,1.41,76.982,Lojas Colombo
316,buraydah,0,SA,1583330063,27,26.33,43.98,298.15,3.60,77.270,Mövenpick Hotel Qassim


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{HotelName}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
coordinates = hotel_df[["Lat","Lng"]]
markers = gmaps.marker_layer(coordinates, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))